<a href="https://colab.research.google.com/github/mutairimjd/ML_Challenges/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2020-10-15 08:49:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip      [      <=>           ]  24.88M   348KB/s    in 75s     

2020-10-15 08:51:15 (339 KB/s) - ‘book-crossings.zip’ saved [26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

#visualising books
#df_books = df_books.drop(columns=['author'])
print(df_books)

              isbn  ...                author
0       0195153448  ...    Mark P. O. Morford
1       0002005018  ...  Richard Bruce Wright
2       0060973129  ...          Carlo D'Este
3       0374157065  ...      Gina Bari Kolata
4       0393045218  ...       E. J. W. Barber
...            ...  ...                   ...
271374  0440400988  ...        Paula Danziger
271375  0525447644  ...            Teri Sloat
271376  006008667X  ...      Christine Wicker
271377  0192126040  ...                 Plato
271378  0767409752  ...   Christopher  Biffle

[271379 rows x 3 columns]


In [ ]:
#visualising ratings
print(df_ratings)

           user         isbn  rating
0        276725   034545104X     0.0
1        276726   0155061224     5.0
2        276727   0446520802     0.0
3        276729   052165615X     3.0
4        276729   0521795028     6.0
...         ...          ...     ...
1149775  276704   1563526298     9.0
1149776  276706   0679447156     0.0
1149777  276709   0515107662    10.0
1149778  276721   0590442449    10.0
1149779  276723  05162443314     8.0

[1149780 rows x 3 columns]


In [ ]:
#remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
counts1 = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(counts1[counts1 >= 200].index)]
counts = df_ratings['rating'].value_counts()
df_ratings = df_ratings[df_ratings['rating'].isin(counts[counts >= 100].index)]

print(df_ratings.shape)

(527556, 3)


In [ ]:
#merging the two dataframes based on isbn inorder to align title and rating
combine_book_rating = pd.merge(df_ratings, df_books, on='isbn')
combine_book_rating = combine_book_rating.drop(columns=['author'])
combine_book_rating.head()

,user,isbn,rating,title
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...


In [ ]:
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['title'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
book_ratingCount.head()

,title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [ ]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,user,isbn,rating,title,totalRatingCount
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,82
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,82
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,82
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,82
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,82


In [ ]:
#popularity_threshold = 50
#rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
#rating_popular_book.head()

rating_popular_book = rating_with_totalRatingCount

In [ ]:
books = rating_popular_book.drop_duplicates(['user', 'title'])
books_pivot = books.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
books_matrix = csr_matrix(books_pivot.values)

In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(books_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = model_knn.kneighbors(books_pivot.loc[book, :].values.reshape(1, -1), n_neighbors = 6)
  reco = []

  for i in range(0, len(distances.flatten())):
      if i == 0:
        None
      else:
        reco.append([books_pivot.index[indices.flatten()[i]], distances.flatten()[i]])

  return [book,reco]

Use the cell below to test your function. The test_book_recommendation() function will inform you if you passed the challenge or need to keep trying.



In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Blue Diary', 0.71828747], ['The Lovely Bones: A Novel', 0.7234864], ['Dry: A Memoir', 0.7510074], ['I Know This Much Is True', 0.7518421], ['Errands', 0.75332904]]]
You havn't passed yet. Keep trying!
